# Train ML model to correct predictions of week 3-4 & 5-6

This notebook create a Machine Learning `ML_model` to predict weeks 3-4 & 5-6 based on `S2S` weeks 3-4 & 5-6 forecasts and is compared to `CPC` observations for the [`s2s-ai-challenge`](https://s2s-ai-challenge.github.io/).

# Synopsis

## Method: `mean bias reduction`

- calculate the mean bias from 2000-2019 deterministic ensemble mean forecast
- remove that mean bias from 2020 forecast deterministic ensemble mean forecast
- no Machine Learning used here

## Data used

type: renku datasets

Training-input for Machine Learning model:
- hindcasts of models:
    - ECMWF: `ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr`

Forecast-input for Machine Learning model:
- real-time 2020 forecasts of models:
    - ECMWF: `ecmwf_forecast-input_2020_biweekly_deterministic.zarr`

Compare Machine Learning model forecast against against ground truth:
- `CPC` observations:
    - `hindcast-like-observations_biweekly_deterministic.zarr`
    - `forecast-like-observations_2020_biweekly_deterministic.zarr`

## Resources used
for training, details in reproducibility

- platform: MPI-M supercompute 1 Node
- memory: 64 GB
- processors: 36 CPU
- storage required: 10 GB

## Safeguards

All points have to be [x] checked. If not, your submission is invalid.

Changes to the code after submissions are not possible, as the `commit` before the `tag` will be reviewed.
(Only in exceptions and if previous effort in reproducibility can be found, it may be allowed to improve readability and reproducibility after November 1st 2021.)

### Safeguards to prevent [overfitting](https://en.wikipedia.org/wiki/Overfitting?wprov=sfti1) 

If the organizers suspect overfitting, your contribution can be disqualified.

  - [x] We didnt use 2020 observations in training (explicit overfitting and cheating)
  - [x] We didnt repeatedly verify my model on 2020 observations and incrementally improved my RPSS (implicit overfitting)
  - [x] We provide RPSS scores for the training period with script `skill_by_year`, see in section 6.3 `predict`.
  - [x] We tried our best to prevent [data leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning)?wprov=sfti1).
  - [x] We honor the `train-validate-test` [split principle](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets). This means that the hindcast data is split into `train` and `validate`, whereas `test` is withheld.
  - [x] We did use `test` explicitly in training or implicitly in incrementally adjusting parameters.
  - [x] We considered [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)).

### Safeguards for Reproducibility
Notebook/code must be independently reproducible from scratch by the organizers (after the competition), if not possible: no prize
  - [x] All training data is publicly available (no pre-trained private neural networks, as they are not reproducible for us)
  - [x] Code is well documented, readable and reproducible.
  - [x] Code to reproduce training and predictions is preferred to run within a day on the described architecture. If the training takes longer than a day, please justify why this is needed. Please do not submit training piplelines, which take weeks to train.

# Imports

In [1]:
import xarray as xr
xr.set_options(display_style='text')

# Get training data

preprocessing of input data may be done in separate notebook/script

## Hindcast

get weekly initialized hindcasts

In [2]:
# preprocessed as renku dataset
!renku storage pull ../data/ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr

In [2]:
hind_2000_2019 = xr.open_zarr("../data/ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr", consolidated=True)

In [ ]:
# preprocessed as renku dataset
!renku storage pull ../data/ecmwf_forecast-input_2020_biweekly_deterministic.zarr

In [3]:
fct_2020 = xr.open_zarr("../data/ecmwf_forecast-input_2020_biweekly_deterministic.zarr", consolidated=True)

## Observations
corresponding to hindcasts

In [ ]:
# preprocessed as renku dataset
!renku storage pull ../data/hindcast-like-observations_2000-2019_biweekly_deterministic.zarr

In [4]:
obs_2000_2019 = xr.open_zarr("../data/hindcast-like-observations_2000-2019_biweekly_deterministic.zarr", consolidated=True)

In [ ]:
# preprocessed as renku dataset
!renku storage pull ../data/forecast-like-observations_2020_biweekly_deterministic.zarr

In [5]:
obs_2020 = xr.open_zarr("../data/forecast-like-observations_2020_biweekly_deterministic.zarr", consolidated=True)

# no ML model

Here, we just remove the mean bias from the ensemble mean forecast.

In [6]:
bias_2000_2019 = (hind_2000_2019.mean('realization') - obs_2000_2019).groupby('forecast_time.weekofyear').mean().compute()

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/xarray/core/accessor_dt.py:381: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  FutureWarning,
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


## `predict`

Create predictions and print `mean(variable, lead_time, longitude, weighted latitude)` RPSS for all years as calculated by `skill_by_year`.

In [7]:
from scripts import make_probabilistic

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/climetlab/plotting/drivers/magics/actions.py:36: UserWarning: Magics library could not be found
  warnings.warn(str(e))


In [ ]:
!renku storage pull ../data/hindcast-like-observations_2000-2019_biweekly_tercile-edges.nc

In [10]:
tercile_file = f'../data/hindcast-like-observations_2000-2019_biweekly_tercile-edges.nc'
tercile_edges = xr.open_dataset(tercile_file)

In [11]:
def create_predictions(fct, bias):
    preds = fct - bias.sel(weekofyear=fct.forecast_time.dt.weekofyear)
    preds = make_probabilistic(preds, tercile_edges)
    return preds.astype('float32')

### `predict` training period in-sample

In [ ]:
!renku storage pull ../data/forecast-like-observations_2020_biweekly_terciled.nc

In [ ]:
!renku storage pull ../data/hindcast-like-observations_2000-2019_biweekly_terciled.zarr

In [12]:
preds_is = create_predictions(hind_2000_2019, bias_2000_2019).compute()

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/xarray/core/accessor_dt.py:381: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  FutureWarning,
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/xarray/core/accessor_dt.py:381: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  FutureWarning,


In [13]:
from scripts import skill_by_year

skill_by_year(preds_is)

,RPSS
year,
2000,-0.141857
2001,-0.203405
2002,-0.202549
2003,-0.206234
2004,-0.549463
2005,-0.168421
2006,-0.184515
2007,-0.616939
2008,-0.195251


### `predict` test

In [14]:
preds_test = create_predictions(fct_2020, bias_2000_2019)

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/xarray/core/accessor_dt.py:381: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  FutureWarning,
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/xarray/core/accessor_dt.py:381: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  FutureWarning,


In [15]:
skill_by_year(preds_test)

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


,RPSS
year,
2020,-0.093422


# Submission

In [16]:
from scripts import assert_predictions_2020
assert_predictions_2020(preds_test)

In [17]:
preds_test.attrs = {'author': 'Aaron Spring', 'author_email': 'aaron.spring@mpimet.mpg.de',
               'comment': 'created for the s2s-ai-challenge as a template for the website',
               'notebook': 'mean_bias_reduction.ipynb',
               'website': 'https://s2s-ai-challenge.github.io/#evaluation'}

html_repr = xr.core.formatting_html.dataset_repr(preds_test)

with open('submission_template_repr.html', 'w') as myFile:
    myFile.write(html_repr)

In [18]:
preds_test.to_netcdf('../submissions/ML_prediction_2020.nc')

In [ ]:
#!git add ../submissions/ML_prediction_2020.nc
#!git add mean_bias_reduction.ipynb

In [ ]:
#!git commit -m "template_test no ML mean bias reduction" # whatever message you want

In [ ]:
#!git tag "submission-no_ML_mean_bias_reduction-0.0.1" # if this is to be checked by scorer, only the last submitted==tagged version will be considered

In [ ]:
#!git push --tags

# Reproducibility

## memory

In [18]:
# https://phoenixnap.com/kb/linux-commands-check-memory-usage
!free -g

             total       used       free     shared    buffers     cached
Mem:            62         15         46          0          0          5
-/+ buffers/cache:         10         52
Swap:            0          0          0


## CPU

In [19]:
!lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                72
On-line CPU(s) list:   0-71
Thread(s) per core:    2
Core(s) per socket:    18
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 79
Model name:            Intel(R) Xeon(R) CPU E5-2695 v4 @ 2.10GHz
Stepping:              1
CPU MHz:               2100.000
BogoMIPS:              4190.01
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              46080K
NUMA node0 CPU(s):     0-17,36-53
NUMA node1 CPU(s):     18-35,54-71


## software

In [20]:
!conda list

# packages in environment at /work/mh0727/m300524/conda-envs/s2s-ai:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_tflow_select             2.3.0                       mkl  
absl-py                   0.12.0           py37h06a4308_0  
aiobotocore               1.2.2              pyhd3eb1b0_0  
aiohttp                   3.7.4            py37h27cfd23_1  
aioitertools              0.7.1              pyhd3eb1b0_0  
anyio                     2.2.0                    pypi_0    pypi
appdirs                   1.4.4                      py_0  
argcomplete               1.12.2                   pypi_0    pypi
argon2-cffi               20.1.0           py37h27cfd23_1  
asciitree                 0.3.3                      py_2  
astunparse                1.6.3                      py_0  
async-timeout             3.0.1            py37h06a4308_0  
async_generator           1.10             py37h28b3542_0  
attrs     